In [107]:
import pandas as pd
import os
import networkx as nx
import numpy as np
pd.__version__

'0.20.3'

# Defining the different data path

In [108]:
# Data path containing all of the raw and processed data
data = '../../../Paper/Data/'
# Result path containing all the results from the analysisi
resultpath = '../../../Paper/Results/'
# ID for the PPI which year it's from 
PPI_ID = "2017_08"

# Loading all the tables

## Loading the raw counts

In [109]:
raw_hipo_fec = pd.read_csv(os.path.join(data,'Screen/Raw_EggLaying_HpoRNAi.csv'))
raw_hipo_ova = pd.read_csv(os.path.join(data,'Screen/Raw_Ova_HpoRNAi.csv'))
raw_xRNAi_fec = pd.read_csv(os.path.join(data,'Screen/Raw_EggLaying.csv'))
hipo_ova = pd.read_csv(os.path.join(data,'Screen/hipo_ova_clean.csv'))
xRNAi_fec = pd.read_csv(os.path.join(data,'Screen/xRNAi_fec_clean.csv'))
hipo_fec = pd.read_csv(os.path.join(data,'Screen/hipo_fec_clean.csv'))

In [110]:
# Calculate the mean for all datasets
mean_ova_gene = hipo_ova.groupby(['FbID'], as_index=False).mean()
mean_fec_gene = hipo_fec.groupby(['FbID', 'Condition'], as_index=False).mean()
mean_xRNAi_gene = xRNAi_fec.groupby(['FbID', 'Condition'], as_index=False).mean()

# Calculate the std for ovariole number (only because the other datasets have only 1 measurement)
std_ova_gene = hipo_ova.groupby(['FbID']).std().reset_index()

In [111]:
# Here we select all the genes that were tested in the screen,
# because the first screen was Hipo RNAi EggLaying measurement, this dataset contains all the tested genes
screen_genes = mean_fec_gene['FbID'].unique()

## Loading gene names

In [112]:
names = pd.read_table(os.path.join(data,'GeneName.csv'))

## Loading the signaling pathway metadata

In [113]:
signaling = pd.read_csv(os.path.join(data,'signaling.csv'))

## Loading the PPI network

In [114]:
G = nx.read_graphml(os.path.join(data,'PPI_{}.graphml'.format(PPI_ID)))

## Loading the networks modules

In [115]:
# Modules computed in the notebook file: Seed-Connector
ova_module_G = nx.read_graphml(os.path.join(resultpath,'Ova_module_{}.graphml'.format(PPI_ID)))
fec_module_G = nx.read_graphml(os.path.join(resultpath,'Hpo_EggL_module_{}.graphml'.format(PPI_ID)))
xRNAi_module_G = nx.read_graphml(os.path.join(resultpath,'EggL_module_{}.graphml'.format(PPI_ID)))
core_module_G = nx.read_graphml(os.path.join(resultpath,'Core_module_{}.graphml'.format(PPI_ID)))

# The list of connector genes
connectors= pd.read_csv(os.path.join(resultpath,"ConnectorGeneList_{}.csv".format(PPI_ID)))

In [116]:
# Grab the list of genes for each modules
ova_module = ova_module_G.nodes()
fec_module = fec_module_G.nodes()
xRNAi_module = xRNAi_module_G.nodes()
core_module = core_module_G.nodes()

## Loading the network metrics

In [117]:
betweenness = pd.read_csv(os.path.join(data, "ScreenPPI_Betweenness.csv"))
closeness = pd.read_csv(os.path.join(data, "ScreenPPI_Closeness.csv"))
eigenvector = pd.read_csv(os.path.join(data, "ScreenPPI_Eigenvector.csv"))
degrees_cen = pd.read_csv(os.path.join(data, "ScreenPPI_DegreeCentrality.csv"))

# Creating the table

## Step 1: Make the list of genes

In [118]:
table = pd.DataFrame(screen_genes, columns=['FbID'])

In [119]:
table = table.merge(raw_hipo_fec[['FbID', 'Condition']], how='left', on='FbID')
table = table.rename(columns={'Condition':'CG number'})

In [120]:
table = table.merge(names, how='left', on='FbID')

## Step 2: Add the screen data for each genes

### Hippo RNAi Egg Laying

In [121]:
# Hippo RNAi Egg Laying screen egg counts
# First we merge the existing table using the FbID column for 1 to 1 matching.
# Then we rename that collumn for a unique name in the global database table
# Rinse and repeat for all values

table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 1'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_1_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_1_Egg_Count'})
table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 2 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_2_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_2_Egg_Count'})
table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 3'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_3_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_3_Egg_Count'})
table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 4 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_4_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_4_Egg_Count'})
table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Day 5'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Day_5_Egg_Zscore', 'Count':'HippoRNAi_EggL_Day_5_Egg_Count'})

table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Sum'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'HippoRNAi_EggL_Sum_Egg_Zscore', 'Count':'HippoRNAi_EggL_Sum_Egg_Count'})

table = table.merge(mean_fec_gene[mean_fec_gene['Condition'] == 'Sum'][['FbID', 'Batch']], how='left', on='FbID')
table = table.rename(columns={'Batch':'HippoRNAi_EggL_Batch'})

# TO DO ADD CONNECTOR WHEN SCREEN IS DONE

### Egg Laying

In [122]:
# Egg Laying screen egg counts
# We use the same technic as above
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 1'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_1_Egg_Zscore', 'Count':'EggL_Day_1_Egg_Count'})
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 2 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_2_Egg_Zscore', 'Count':'EggL_Day_2_Egg_Count'})
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 3'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_3_Egg_Zscore', 'Count':'EggL_Day_3_Egg_Count'})
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 4 '][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_4_Egg_Zscore', 'Count':'EggL_Day_4_Egg_Count'})
table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Day 5'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Day_5_Egg_Zscore', 'Count':'EggL_Day_5_Egg_Count'})

table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Sum'][['FbID', 'Count', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Z':'EggL_Sum_Egg_Zscore', 'Count':'EggL_Sum_Egg_Count'})

table = table.merge(mean_xRNAi_gene[mean_xRNAi_gene['Condition'] == 'Sum'][['FbID','Batch']], how='left', on='FbID')
table = table.rename(columns={'Batch':'EggL_Batch'})

# TO DO ADD CONNECTOR WHEN SCREEN IS DONE

### Ovariole Number

In [123]:
# Hippo RNAi Ovariole number counts screen
# We can merge all the data directly because we do not have many conditions in this table
table = table.merge(mean_ova_gene[['FbID', 'Batch', 'OvarioleNb', 'Z']], how='left', on='FbID')
table = table.rename(columns={'Batch':'HippoRNAi_Ova_Batch', 
                              'OvarioleNb':'HippoRNAi_Ova_OvarioleNb_Mean', 
                              'Z':'HippoRNAi_Ova_OvarioleNb_Mean_Zscore'})

table = table.merge(std_ova_gene[['FbID', 'OvarioleNb']], how='left', on='FbID')
table = table.rename(columns={'OvarioleNb':'HippoRNAi_Ova_OvarioleNb_Std'})

In [124]:
# But we want to add the raw data to the table, so we extract the 20 columns from the raw counts
raw = raw_hipo_ova[raw_hipo_ova['FbID'].notnull()][['Fly 1', 'Fly 1.1', 'Fly 2', 'Fly 2.1', 
                                            'Fly 3', 'Fly 3.1', 'Fly 4', 'Fly 4.1', 
                                            'Fly 5', 'Fly 5.1', 'Fly 6', 'Fly 6.1',
                                            'Fly 7', 'Fly 7.1', 'Fly 8', 'Fly 8.1',
                                            'Fly 9','Fly 9.1', 'Fly 10', 'Fly 10.1',
                                                 'FbID']].groupby(['FbID']).mean().reset_index()

In [125]:
# We merge them
table = table.merge(raw, how='left', on='FbID')


In [126]:
# And then we rename them to follow the naming scheme
table = table.rename(columns={'Fly 1'   : "HippoRNAi_Ova_OvarioleNb_Fly_1.1",
                              'Fly 1.1' : "HippoRNAi_Ova_OvarioleNb_Fly_1.2",
                              'Fly 2'   : "HippoRNAi_Ova_OvarioleNb_Fly_2.1",
                              'Fly 2.1' : "HippoRNAi_Ova_OvarioleNb_Fly_2.2",
                              'Fly 3'   : "HippoRNAi_Ova_OvarioleNb_Fly_3.1",
                              'Fly 3.1' : "HippoRNAi_Ova_OvarioleNb_Fly_3.2",
                              'Fly 4'   : "HippoRNAi_Ova_OvarioleNb_Fly_4.1",
                              'Fly 4.1' : "HippoRNAi_Ova_OvarioleNb_Fly_4.2",
                              'Fly 5'   : "HippoRNAi_Ova_OvarioleNb_Fly_5.1",
                              'Fly 5.1' : "HippoRNAi_Ova_OvarioleNb_Fly_5.2",
                              'Fly 6'   : "HippoRNAi_Ova_OvarioleNb_Fly_6.1",
                              'Fly 6.1' : "HippoRNAi_Ova_OvarioleNb_Fly_6.2",
                              'Fly 7'   : "HippoRNAi_Ova_OvarioleNb_Fly_7.1",
                              'Fly 7.1' : "HippoRNAi_Ova_OvarioleNb_Fly_7.2",
                              'Fly 8'   : "HippoRNAi_Ova_OvarioleNb_Fly_8.1",
                              'Fly 8.1' : "HippoRNAi_Ova_OvarioleNb_Fly_8.2",
                              'Fly 9'   : "HippoRNAi_Ova_OvarioleNb_Fly_9.1",
                              'Fly 9.1' : "HippoRNAi_Ova_OvarioleNb_Fly_9.2",
                              'Fly 10'  : "HippoRNAi_Ova_OvarioleNb_Fly_10.1",
                              'Fly 10.1': "HippoRNAi_Ova_OvarioleNb_Fly_10.2"
                             })

# TO DO ADD CONNECTOR WHEN SCREEN IS DONE

## Step 3: Adding the network metrics

In [127]:
# Merge and rename as above
table = table.merge(betweenness, how='left', on='FbID')
table = table.rename(columns={'Betweeness':'PPI_betweenness_centrality'})

In [128]:
table = table.merge(closeness, how='left', on='FbID')
table = table.rename(columns={'Closeness':'PPI_closeness_centrality'})

In [129]:
table = table.merge(eigenvector, how='left', on='FbID')
table = table.rename(columns={'EigenVector':'PPI_eigenvector_centrality'})

In [130]:
table = table.merge(degrees_cen, how='left', on='FbID')
table = table.rename(columns={'DegreeC':'PPI_degree_centrality'})

## Step 4: Adding the newtork modules

In [131]:
table['FbID'].isin(ova_module).sum()

70

In [132]:
# We create 4 columns, where if a gene is found in a module it is 1 if not a 0
# np.where is key here, if condition == true, then X, else Y 
table['HippoRNAi_Ova_Module'] = np.where(table['FbID'].isin(ova_module), 1, 0)
table['HippoRNAi_EggL_Module'] = np.where(table['FbID'].isin(fec_module), 1, 0)
table['EggL_Module'] = np.where(table['FbID'].isin(xRNAi_module), 1, 0)
table['Core_Module'] = np.where(table['FbID'].isin(core_module), 1, 0)

## Step 5: Adding Signaling pathways

In [133]:
# We need to make this tidy data, so we need to add one column per signaling patway, with a 0 or a 1 
# We iterate over all signaling pathway and add a column for each with 1 and 0s using the same np.where technic as above
# But we first need to make the list of FbID that have this signaling pathway 
# that is: pathway_genes = signaling[signaling['Sig'] == pathway]['FbID']
for pathway in signaling['Sig'].unique():
    pathway_genes = signaling[signaling['Sig'] == pathway]['FbID']
    table['{}_pathway'.format(pathway)] = np.where(table['FbID'].isin(pathway_genes), 1, 0)

## Step 6: Adding Connector genes

In [134]:
table['Connector_Gene'] = np.where(table['FbID'].isin(connectors['FbID']), 1, 0)

# Asserting that the data is correctly entered

In [135]:
# Test that the number of genes in the database is equal to the number of gene screened in the primary screen
assert(len(table) == len(screen_genes))

In [136]:
pd.__version__

'0.20.3'

In [100]:
# Test that the number of values in each screen correspond to the raw table for each collumn
# We iterate over all the collumns
# For each collumns we define a test
# try the assertion
# if wrong then print an error message
for column in table.columns:
    if 'Module' in column:
        try:
            assert(len(table[table[column].notna()]) == len(screen_genes))
        except:
            print("Discrepancy in column: {}".format(column))
    elif '_pathway' in column:
        try:
            assert(len(table[table[column].notna()]) == len(screen_genes))
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'HippoRNAi_EggL' in column:
        try:
            assert(len(table[table[column].notna()]) == len(mean_fec_gene['FbID'].unique()))
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'EggL' in column:
        try:
            assert(len(table[table[column].notna()]) == len(mean_xRNAi_gene['FbID'].unique()))
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'HippoRNAi_Ova' in column:
        try:
            assert(len(table[table[column].notna()]) == len(mean_ova_gene['FbID'].unique()))
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'Connector_Gene' == column:
        try:
            assert(len(table[table[column] == 1]) == len(connectors['FbID'].unique()))
        except:
            print("Discrepancy in column: {}".format(column))
    elif 'PPI_' in column:
        try:
            assert(len(table[table[column].notna()]) ==  len(table[table['FbID'].isin(G.nodes())]))
        except:
            print("Discrepancy in column: {}".format(column))
    elif "CG" in column:
        try:
            assert(len(table[table[column].notna()]) == len(screen_genes))
        except:
            print("Discrepancy in column: {}".format(column))
            

Discrepancy in column: CG number
Discrepancy in column: HippoRNAi_EggL_Day_1_Egg_Count
Discrepancy in column: HippoRNAi_EggL_Day_1_Egg_Zscore
Discrepancy in column: HippoRNAi_EggL_Day_2_Egg_Count
Discrepancy in column: HippoRNAi_EggL_Day_2_Egg_Zscore
Discrepancy in column: HippoRNAi_EggL_Day_3_Egg_Count
Discrepancy in column: HippoRNAi_EggL_Day_3_Egg_Zscore
Discrepancy in column: HippoRNAi_EggL_Day_4_Egg_Count
Discrepancy in column: HippoRNAi_EggL_Day_4_Egg_Zscore
Discrepancy in column: HippoRNAi_EggL_Day_5_Egg_Count
Discrepancy in column: HippoRNAi_EggL_Day_5_Egg_Zscore
Discrepancy in column: HippoRNAi_EggL_Sum_Egg_Count
Discrepancy in column: HippoRNAi_EggL_Sum_Egg_Zscore
Discrepancy in column: HippoRNAi_EggL_Batch
Discrepancy in column: EggL_Day_1_Egg_Count
Discrepancy in column: EggL_Day_1_Egg_Zscore
Discrepancy in column: EggL_Day_2_Egg_Count
Discrepancy in column: EggL_Day_2_Egg_Zscore
Discrepancy in column: EggL_Day_3_Egg_Count
Discrepancy in column: EggL_Day_3_Egg_Zscore
Discre

In [137]:
table.columns

Index(['FbID', 'CG number', 'NAME', 'SYMBOL', 'HippoRNAi_EggL_Day_1_Egg_Count',
       'HippoRNAi_EggL_Day_1_Egg_Zscore', 'HippoRNAi_EggL_Day_2_Egg_Count',
       'HippoRNAi_EggL_Day_2_Egg_Zscore', 'HippoRNAi_EggL_Day_3_Egg_Count',
       'HippoRNAi_EggL_Day_3_Egg_Zscore', 'HippoRNAi_EggL_Day_4_Egg_Count',
       'HippoRNAi_EggL_Day_4_Egg_Zscore', 'HippoRNAi_EggL_Day_5_Egg_Count',
       'HippoRNAi_EggL_Day_5_Egg_Zscore', 'HippoRNAi_EggL_Sum_Egg_Count',
       'HippoRNAi_EggL_Sum_Egg_Zscore', 'HippoRNAi_EggL_Batch',
       'EggL_Day_1_Egg_Count', 'EggL_Day_1_Egg_Zscore', 'EggL_Day_2_Egg_Count',
       'EggL_Day_2_Egg_Zscore', 'EggL_Day_3_Egg_Count',
       'EggL_Day_3_Egg_Zscore', 'EggL_Day_4_Egg_Count',
       'EggL_Day_4_Egg_Zscore', 'EggL_Day_5_Egg_Count',
       'EggL_Day_5_Egg_Zscore', 'EggL_Sum_Egg_Count', 'EggL_Sum_Egg_Zscore',
       'EggL_Batch', 'HippoRNAi_Ova_Batch', 'HippoRNAi_Ova_OvarioleNb_Mean',
       'HippoRNAi_Ova_OvarioleNb_Mean_Zscore', 'HippoRNAi_Ova_OvarioleNb_Std

# Saving the table

In [138]:
table.to_csv(os.path.join(resultpath, "MasterTable.csv"), index=False)